In [1]:
import  os
import  nibabel             as nib
import  pandas              as pd
import  numpy               as np
import  matplotlib.pyplot   as plt
from    skimage.io          import imread
import  re
import  seaborn             as sns

from niftynet.io.image_reader                                   import ImageReader
from niftynet.contrib.sampler_pairwise.sampler_pairwise_uniform import PairwiseUniformSampler
from niftynet.layer.pad                                         import PadLayer
from niftynet.layer.rand_elastic_deform                         import RandomElasticDeformationLayer
from niftynet.layer.mean_variance_normalisation                 import MeanVarNormalisationLayer
from niftynet.layer.rand_flip                                   import RandomFlipLayer

def plot_slides(images, figsize=(10,5)):
    f, axes = plt.subplots(2,3, figsize=figsize)
    for i, slice_id in enumerate(images):
        axes[i][0].imshow(images[slice_id]['img'], cmap='gray')
        axes[i][0].set_title('Image %s' % slice_id)
        axes[i][1].imshow(images[slice_id]['seg'], cmap='gray')
        axes[i][1].set_title('Segmentation %s' % slice_id)
        axes[i][2].imshow(images[slice_id]['weight'], cmap='jet', vmin=0, vmax=10)
        axes[i][2].set_title('Weight Map %s' % slice_id)

        for ax in axes[i]:
            ax.set_axis_off()
    f.tight_layout()


def grab_demo_images(image_dir, slice_ids, image_prefix_dict):
    images = {slice_id: {
            key: imread(os.path.join(image_dir, image_prefix_dict[key] + '%s.tif' % slice_id))
            for key in image_prefix_dict}
        for slice_id in slice_ids}
    return images

def create_image_reader(num_controlpoints, std_deformation_sigma):
    # creating an image reader.
    data_param = \
        {'cell': {'path_to_search': '/home/dr3m/travaux/traineeship/niftynet_tutorial/demos/models', # PhC-C2DH-U373, DIC-C2DH-HeLa
                'filename_contains': 'data',
                'loader': 'skimage'},
         'label': {'path_to_search': '/home/dr3m/travaux/traineeship/niftynet_tutorial/demos/models', # PhC-C2DH-U373, DIC-C2DH-HeLa
                'filename_contains': 'index',
                'loader': 'skimage',
                'interp_order' : 0}
        }
    reader = ImageReader().initialise(data_param)

    reader.add_preprocessing_layers(MeanVarNormalisationLayer(image_name = 'cell'))

    reader.add_preprocessing_layers(PadLayer(
                     image_name=['cell', 'label'],
                     border=(92,92,0),
                     mode='symmetric'))

    reader.add_preprocessing_layers(RandomElasticDeformationLayer(
                     num_controlpoints=num_controlpoints,
                     std_deformation_sigma=std_deformation_sigma,
                     proportion_to_augment=1,
                     spatial_rank=2))

#     reader.add_preprocessing_layers(RandomFlipLayer(
#                  flip_axes=(0,1)))

    return reader

INFO:tensorflow:TensorFlow version 1.13.1
INFO:tensorflow:Available Image Loaders:
['nibabel', 'opencv', 'skimage', 'pillow', 'simpleitk', 'dummy'].


In [2]:
U373_dir = "./models/model.ckpt-33000"
U373_imgs = grab_demo_images(U373_dir, ['002'], {'img': 'data-', 'seg': 'index', 'weight': 'meta'})

plot_slides(U373_imgs, figsize=(9,5))

FileNotFoundError: [Errno 2] No such file or directory: '/home/dr3m/travaux/traineeship/niftynet_tutorial/demos/models/model.ckpt-33000/data-002.tif'